# Limpieza de Datos en Pandas (data cleansing)

**Nota para mi yo del futuro:**
Es un paso que siempre debe hacerse!!!!
En el mundo real, siguiendo el principio de entropía, siempre está el riesgo de que un dataset contenga datos duplicados o faltantes (o nulos), o directamente errores en la recogida de datos (tipos de datos incorrectos, etc.).

In [1]:
# Script de ejemplo (con un dataset deliberadamente con errores)
from typing import Any
import pandas as pd


data: dict[str, list[Any]] = {
    'product_id': [1001, 1002, 1003, 1003],
    'sold': [30, None, 25, 25],
    'price': [20.5, 15.0, None, 22.5]
}

df: pd.DataFrame = pd.DataFrame(data)
print(df)

   product_id  sold  price
0        1001  30.0   20.5
1        1002   NaN   15.0
2        1003  25.0    NaN
3        1003  25.0   22.5


In [2]:
# Importante usar df.info() para diagnosticar el dataset
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   product_id  4 non-null      int64  
 1   sold        3 non-null      float64
 2   price       3 non-null      float64
dtypes: float64(2), int64(1)
memory usage: 228.0 bytes
None


In [3]:
# Encontrar las celdas nulas (a ojo)
print(df.isnull())

   product_id   sold  price
0       False  False  False
1       False   True  False
2       False  False   True
3       False  False  False


In [4]:
# Encontrar la cantidad de celdas nulas según serie
print(df.isnull().sum())

product_id    0
sold          1
price         1
dtype: int64


**Otra nota para mi yo del futuro**: La forma de lidiar con los registros con datos nulos, repetidos, dañados, etc. varía de caso a caso.

NO HAY FORMA ÚNICA DE RESOLVER ESTE PROBLEMA!!!!!!! 😱

En algunos casos, generalmente en muestras grandes, se pueden sacrificar las instancias con datos nulos, sin afectar mayormente los análisis posteriores. En otros, especialmente con muestras pequeñas, se prefiere conservar todas las instancias, pero reemplazando las celdas faltantes con algún valor por defecto, el cual puede ser fijo o depender de los mismos datos ya existentes.

De cualquier forma, vale la pena abordar el problema con el jefe de proyecto o de investigación para resolver el camino a escoger.

In [5]:
# En el caso de eliminar registros con valores faltantes (NaN)
df_without_nan: pd.DataFrame = df.dropna()
print(df_without_nan)

   product_id  sold  price
0        1001  30.0   20.5
3        1003  25.0   22.5


In [6]:
# En el caso de reemplazar los valores faltantes (NaN) por un valor
default_values_for_nan: dict = {
    'sold': 0,                  # valor fijo
    'price': df['price'].mean() # valor depende de la media de la serie
}
df_replaced_nan: pd.DataFrame = df.fillna(default_values_for_nan)
print(df_replaced_nan)

   product_id  sold      price
0        1001  30.0  20.500000
1        1002   0.0  15.000000
2        1003  25.0  19.333333
3        1003  25.0  22.500000


También es muy importante verificar que el tipo de dato de cada serie sea el más adecuado para cada caso. Véase los ejemplos a continuación:

In [7]:
# La cantidad de productos vendidos debe ser representado como integer, no float
df_replaced_nan['sold'] = df_replaced_nan['sold'].astype('int64')
print(df_replaced_nan)

   product_id  sold      price
0        1001    30  20.500000
1        1002     0  15.000000
2        1003    25  19.333333
3        1003    25  22.500000


Ejemplos de qué hacer con los registros con datos duplicados:

In [8]:
# Eliminar registros con duplicados
df_unique_values: pd.DataFrame = df_replaced_nan.drop_duplicates()
print(df_unique_values)

   product_id  sold      price
0        1001    30  20.500000
1        1002     0  15.000000
2        1003    25  19.333333
3        1003    25  22.500000


**IMPORTANTE:** No hubo eliminación aquí, porque `drop_duplicates()` (sin parámetros) elimina registros cuando dos o más son _completamente_ iguales. En el ejemplo anterior, dos registros comparten los mismos valores para `product_id` y `sold`, pero difieren en `price`.

No obstante, es posible pasarle parámetros a `drop_duplicates()` para nuestros intereses:

In [9]:
# Eliminar registros con duplicados (con parámetros)
df_unique_values: pd.DataFrame = df_replaced_nan.drop_duplicates(subset=['product_id'])
print(df_unique_values)

   product_id  sold      price
0        1001    30  20.500000
1        1002     0  15.000000
2        1003    25  19.333333
